In [1]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.2 MB/s eta 0:00:00


###Here we are importing the all the required Libraries

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import re
import torch
from transformers import AutoTokenizer, AutoModel
import contractions
import numpy as np


In [39]:
df= pd.read_csv("/content/drive/MyDrive/EMR/patient_post_op_complications.csv")

###We are starting to see what all is present in the dataset like the datatype of each column what all columns are present and how many null values are present

In [40]:
df

,LOG_ID,MRN,Element_Name,CONTEXT_NAME,Element_abbr,SMRTDTA_ELEM_VALUE
0,b00c588406aae5cc,1a42f0bed60c5598,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN
1,a16571d21874c0a7,596a6dc60f54a0bb,AN AQI POST-OP COMPLICATIONS,NOTE,AN Post-op Complications,NaN
2,a16571d21874c0a7,596a6dc60f54a0bb,AN AQI POST-OP COMPLICATIONS,NOTE,AN Post-op Complications,NaN
3,1d9ff2d157fa4c86,60e6ea6cf06b58d7,AN AQI POST-OP COMPLICATIONS,ORDER,AN Post-op Complications,NaN
4,2ae43a4a39735956,59c5483e0654e105,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,Other
...,...,...,...,...,...,...
203940,41e9d0fc85705586,a7b31506e6207021,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN
203941,c998b4d14118fdf8,3f1772becaaebb81,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN
203942,689a22b76d123ff6,65c32dd7fd5e35e7,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN
203943,58a3296cc7491dd2,f7fc111bcf19de88,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203945 entries, 0 to 203944
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   LOG_ID              203945 non-null  object
 1   MRN                 203945 non-null  object
 2   Element_Name        203945 non-null  object
 3   CONTEXT_NAME        203945 non-null  object
 4   Element_abbr        203945 non-null  object
 5   SMRTDTA_ELEM_VALUE  6741 non-null    object
dtypes: object(6)
memory usage: 9.3+ MB


In [42]:
df.describe()

,LOG_ID,MRN,Element_Name,CONTEXT_NAME,Element_abbr,SMRTDTA_ELEM_VALUE
count,203945,203945,203945,203945,203945,6741
unique,64214,36985,12,3,12,514
top,cedf2d18ee468c8d,1bb09d5761661c7d,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,Other
freq,52,1464,200139,174665,200139,895


###As we can see that Post-op complications are repeating in every unique values of the column we are removing it and making it more readable

In [43]:
df['Element_Name'].value_counts()

,count
Element_Name,
AN AQI POST-OP COMPLICATIONS,200139
ANE POST OP COMPLICATION OTHER,1094
AN POST-OP COMPLICATION CARDIOVASCULAR,866
ANE POST OP COMPLICATION RESPIRATORY,748
AN POST-OP COMPLICATION AIRWAY,375
AN POST-OP COMPLICATION METABOLIC,154
AN POST-OP COMPLICATION NEUROLOGICAL,147
AN POST-OP COMPLICATION ADMINISTRATIVE,118
AN POST-OP COMPLICATION INJURY/INFECTION,117


In [44]:
df['Post_OP_type'] = df['Element_Name'].replace({'POST-OP COMPLICATIONS': '', 'POST-OP COMPLICATION': '',"POST OP COMPLICATION":""}, regex=True)

In [45]:
df['Post_OP_type'].value_counts()

,count
Post_OP_type,
AN AQI,200139
ANE OTHER,1094
AN CARDIOVASCULAR,866
ANE RESPIRATORY,748
AN AIRWAY,375
AN METABOLIC,154
AN NEUROLOGICAL,147
AN ADMINISTRATIVE,118
AN INJURY/INFECTION,117


####Now it is more readable

###Here we are checking the number of null value present in the SMRTDTA_ELEM_VALUE column



In [46]:
df['SMRTDTA_ELEM_VALUE'].isnull().sum()

197204

In [47]:
df['SMRTDTA_ELEM_VALUE']

,SMRTDTA_ELEM_VALUE
0,NaN
1,NaN
2,NaN
3,NaN
4,Other
...,...
203940,NaN
203941,NaN
203942,NaN
203943,NaN


In [48]:
df['SMRTDTA_ELEM_VALUE'].isnull().sum()

197204

###Label encoding the given CONTEXT_NAME column unique labels

In [49]:
df['CONTEXT_NAME'].unique()

array(['ENCOUNTER', 'NOTE', 'ORDER'], dtype=object)

In [50]:
df['CONTEXT_NAME'].value_counts()


,count
CONTEXT_NAME,
ENCOUNTER,174665
ORDER,16071
NOTE,13209


In [51]:
label_encoder = LabelEncoder()

# Fit and transform the column
df['CONTEXT_NAME_encoded'] = label_encoder.fit_transform(df['CONTEXT_NAME'])


### Performing One-hot encoding on the column Element_abbr

In [52]:
df['Element_abbr'].unique()


array(['AN Post-op Complications', 'Airway', 'Respiratory', 'Other',
       'Cardiovascular', 'Neurological', 'Regional', 'Administrative',
       'Injury/Infection', 'Medication', 'Metabolic', 'Chronic Pain'],
      dtype=object)

In [53]:
encoder = OneHotEncoder(sparse_output=False)  # sparse=False ensures that it returns a dense array

# Fit and transform the data
encoded_columns = encoder.fit_transform(df[['Post_OP_type']])

# Create a new DataFrame with the one-hot encoded columns
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['Post_OP_type']))

# Concatenate the original DataFrame with the encoded DataFrame
df_final = pd.concat([df, encoded_df], axis=1)
df_final[encoder.get_feature_names_out(['Post_OP_type'])] = df_final[encoder.get_feature_names_out(['Post_OP_type'])].astype(int)


In [54]:
df_final

,LOG_ID,MRN,Element_Name,CONTEXT_NAME,Element_abbr,SMRTDTA_ELEM_VALUE,Post_OP_type,CONTEXT_NAME_encoded,Post_OP_type_AN ADMINISTRATIVE,Post_OP_type_AN AIRWAY,Post_OP_type_AN CARDIOVASCULAR,Post_OP_type_AN CHRONIC PAIN,Post_OP_type_AN INJURY/INFECTION,Post_OP_type_AN MEDICATION,Post_OP_type_AN METABOLIC,Post_OP_type_AN NEUROLOGICAL,Post_OP_type_AN AQI,Post_OP_type_ANE OTHER,Post_OP_type_ANE REGIONAL,Post_OP_type_ANE RESPIRATORY
0,b00c588406aae5cc,1a42f0bed60c5598,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN,AN AQI,0,0,0,0,0,0,0,0,0,1,0,0,0
1,a16571d21874c0a7,596a6dc60f54a0bb,AN AQI POST-OP COMPLICATIONS,NOTE,AN Post-op Complications,NaN,AN AQI,1,0,0,0,0,0,0,0,0,1,0,0,0
2,a16571d21874c0a7,596a6dc60f54a0bb,AN AQI POST-OP COMPLICATIONS,NOTE,AN Post-op Complications,NaN,AN AQI,1,0,0,0,0,0,0,0,0,1,0,0,0
3,1d9ff2d157fa4c86,60e6ea6cf06b58d7,AN AQI POST-OP COMPLICATIONS,ORDER,AN Post-op Complications,NaN,AN AQI,2,0,0,0,0,0,0,0,0,1,0,0,0
4,2ae43a4a39735956,59c5483e0654e105,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,Other,AN AQI,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203940,41e9d0fc85705586,a7b31506e6207021,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN,AN AQI,0,0,0,0,0,0,0,0,0,1,0,0,0
203941,c998b4d14118fdf8,3f1772becaaebb81,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN,AN AQI,0,0,0,0,0,0,0,0,0,1,0,0,0
203942,689a22b76d123ff6,65c32dd7fd5e35e7,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN,AN AQI,0,0,0,0,0,0,0,0,0,1,0,0,0
203943,58a3296cc7491dd2,f7fc111bcf19de88,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN,AN AQI,0,0,0,0,0,0,0,0,0,1,0,0,0


###Dropping the unwanted columns

In [55]:
df_final = df_final.drop(columns=['Element_Name','CONTEXT_NAME','Element_abbr','Post_OP_type'])


In [56]:
df_final

,LOG_ID,MRN,SMRTDTA_ELEM_VALUE,CONTEXT_NAME_encoded,Post_OP_type_AN ADMINISTRATIVE,Post_OP_type_AN AIRWAY,Post_OP_type_AN CARDIOVASCULAR,Post_OP_type_AN CHRONIC PAIN,Post_OP_type_AN INJURY/INFECTION,Post_OP_type_AN MEDICATION,Post_OP_type_AN METABOLIC,Post_OP_type_AN NEUROLOGICAL,Post_OP_type_AN AQI,Post_OP_type_ANE OTHER,Post_OP_type_ANE REGIONAL,Post_OP_type_ANE RESPIRATORY
0,b00c588406aae5cc,1a42f0bed60c5598,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0
1,a16571d21874c0a7,596a6dc60f54a0bb,NaN,1,0,0,0,0,0,0,0,0,1,0,0,0
2,a16571d21874c0a7,596a6dc60f54a0bb,NaN,1,0,0,0,0,0,0,0,0,1,0,0,0
3,1d9ff2d157fa4c86,60e6ea6cf06b58d7,NaN,2,0,0,0,0,0,0,0,0,1,0,0,0
4,2ae43a4a39735956,59c5483e0654e105,Other,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203940,41e9d0fc85705586,a7b31506e6207021,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0
203941,c998b4d14118fdf8,3f1772becaaebb81,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0
203942,689a22b76d123ff6,65c32dd7fd5e35e7,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0
203943,58a3296cc7491dd2,f7fc111bcf19de88,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0


###Performing Data cleaning on the free text column ('SMRSTA_ELEM_VALUE') and converting it into embedings using tokenizers from  PUB_med fine tuned bert model

In [57]:
# df['SMRTDTA_ELEM_VALUE']= df['SMRTDTA_ELEM_VALUE'].fillna('')
df_final['SMRTDTA_ELEM_VALUE']=  df_final['SMRTDTA_ELEM_VALUE'].fillna('missing')


In [58]:
df_final['SMRTDTA_ELEM_VALUE'] = df_final['SMRTDTA_ELEM_VALUE'].str.lower()


In [59]:
# df['SMRTDTA_ELEM_VALUE'].unique()

In [60]:
df_final['SMRTDTA_ELEM_VALUE'] = df_final['SMRTDTA_ELEM_VALUE'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


In [74]:
df_final['SMRTDTA_ELEM_VALUE'] = df_final['SMRTDTA_ELEM_VALUE'].apply(lambda x: contractions.fix(x))
df_final['SMRTDTA_ELEM_VALUE'] = df_final['SMRTDTA_ELEM_VALUE'].apply(lambda x: x.strip())


In [75]:
df_final['SMRTDTA_tokens'] = df_final['SMRTDTA_ELEM_VALUE'].apply(lambda x: x.split())


In [78]:
def meanpooling(output, mask):
    embeddings = output[0]
    mask = mask.unsqueeze(-1).expand(embeddings.size()).float()
    return torch.sum(embeddings * mask, 1) / torch.clamp(mask.sum(1), min=1e-9)

tokenizer = AutoTokenizer.from_pretrained("neuml/pubmedbert-base-embeddings")
model = AutoModel.from_pretrained("neuml/pubmedbert-base-embeddings")

def get_sentence_embedding(sentence):

    if pd.isnull(sentence) or sentence == 'missing':
        return np.array([])
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        output = model(**inputs)

    embedding = meanpooling(output, inputs['attention_mask'])
    return embedding.squeeze().numpy()

df_final['SMRTDTA_embeddings'] = df_final['SMRTDTA_ELEM_VALUE'].apply(get_sentence_embedding)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [82]:
df_final['SMRTDTA_ELEM_VALUE'].value_counts()

,count
SMRTDTA_ELEM_VALUE,
missing,197204
other,895
cardiovascular,620
respiratory,596
airway,343
...,...
ed with 4mgkg sugammadex 220mg he then had 4 twiches and was breathing on his own after 20 mins of not waking up patients glucose was checked and it was 110 while patient was breathing he had pin point pupils thus we gave 80mcg of narcan pat,1
pt arrived to or intubated without sedation,1
to give another 120mcg of narcan total 200mcg patient woke up immediately he was extubated and taken to pacu,1


In [83]:
print(df_final[['SMRTDTA_ELEM_VALUE', 'SMRTDTA_embeddings']])

       SMRTDTA_ELEM_VALUE                                 SMRTDTA_embeddings
0                 missing                                                 []
1                 missing                                                 []
2                 missing                                                 []
3                 missing                                                 []
4                   other  [-1.0085479, 0.5639785, 0.59146255, -0.6559291...
...                   ...                                                ...
203940            missing                                                 []
203941            missing                                                 []
203942            missing                                                 []
203943            missing                                                 []
203944            missing                                                 []

[203945 rows x 2 columns]


In [84]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203945 entries, 0 to 203944
Data columns (total 18 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   LOG_ID                             203945 non-null  object
 1   MRN                                203945 non-null  object
 2   SMRTDTA_ELEM_VALUE                 203945 non-null  object
 3   CONTEXT_NAME_encoded               203945 non-null  int64 
 4   Post_OP_type_AN  ADMINISTRATIVE    203945 non-null  int64 
 5   Post_OP_type_AN  AIRWAY            203945 non-null  int64 
 6   Post_OP_type_AN  CARDIOVASCULAR    203945 non-null  int64 
 7   Post_OP_type_AN  CHRONIC PAIN      203945 non-null  int64 
 8   Post_OP_type_AN  INJURY/INFECTION  203945 non-null  int64 
 9   Post_OP_type_AN  MEDICATION        203945 non-null  int64 
 10  Post_OP_type_AN  METABOLIC         203945 non-null  int64 
 11  Post_OP_type_AN  NEUROLOGICAL      203945 non-null  

In [85]:
df_final

,LOG_ID,MRN,SMRTDTA_ELEM_VALUE,CONTEXT_NAME_encoded,Post_OP_type_AN ADMINISTRATIVE,Post_OP_type_AN AIRWAY,Post_OP_type_AN CARDIOVASCULAR,Post_OP_type_AN CHRONIC PAIN,Post_OP_type_AN INJURY/INFECTION,Post_OP_type_AN MEDICATION,Post_OP_type_AN METABOLIC,Post_OP_type_AN NEUROLOGICAL,Post_OP_type_AN AQI,Post_OP_type_ANE OTHER,Post_OP_type_ANE REGIONAL,Post_OP_type_ANE RESPIRATORY,SMRTDTA_tokens,SMRTDTA_embeddings
0,b00c588406aae5cc,1a42f0bed60c5598,missing,0,0,0,0,0,0,0,0,0,1,0,0,0,[missing],[]
1,a16571d21874c0a7,596a6dc60f54a0bb,missing,1,0,0,0,0,0,0,0,0,1,0,0,0,[missing],[]
2,a16571d21874c0a7,596a6dc60f54a0bb,missing,1,0,0,0,0,0,0,0,0,1,0,0,0,[missing],[]
3,1d9ff2d157fa4c86,60e6ea6cf06b58d7,missing,2,0,0,0,0,0,0,0,0,1,0,0,0,[missing],[]
4,2ae43a4a39735956,59c5483e0654e105,other,0,0,0,0,0,0,0,0,0,1,0,0,0,[other],"[-1.0085479, 0.5639785, 0.59146255, -0.6559291..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203940,41e9d0fc85705586,a7b31506e6207021,missing,0,0,0,0,0,0,0,0,0,1,0,0,0,[missing],[]
203941,c998b4d14118fdf8,3f1772becaaebb81,missing,0,0,0,0,0,0,0,0,0,1,0,0,0,[missing],[]
203942,689a22b76d123ff6,65c32dd7fd5e35e7,missing,0,0,0,0,0,0,0,0,0,1,0,0,0,[missing],[]
203943,58a3296cc7491dd2,f7fc111bcf19de88,missing,0,0,0,0,0,0,0,0,0,1,0,0,0,[missing],[]


In [86]:
df_final.columns

Index(['LOG_ID', 'MRN', 'SMRTDTA_ELEM_VALUE', 'CONTEXT_NAME_encoded',
       'Post_OP_type_AN  ADMINISTRATIVE', 'Post_OP_type_AN  AIRWAY',
       'Post_OP_type_AN  CARDIOVASCULAR', 'Post_OP_type_AN  CHRONIC PAIN',
       'Post_OP_type_AN  INJURY/INFECTION', 'Post_OP_type_AN  MEDICATION',
       'Post_OP_type_AN  METABOLIC', 'Post_OP_type_AN  NEUROLOGICAL',
       'Post_OP_type_AN AQI ', 'Post_OP_type_ANE  OTHER',
       'Post_OP_type_ANE  REGIONAL', 'Post_OP_type_ANE  RESPIRATORY',
       'SMRTDTA_tokens', 'SMRTDTA_embeddings'],
      dtype='object')

In [ ]:
df_final = df_final.drop(columns=['SMRTDTA_tokens'])


###Saving the preprocessed File for further EDA

In [11]:
df_final.to_csv('/content/drive/MyDrive/EMR/Processed_patient_post_op_complications.csv', index=False)

NameError: name 'df_final' is not defined